In [4]:
import requests
import json
import re
import pandas as pd
from datetime import datetime

today_date = datetime.now().strftime('%Y-%m-%d')
load_name = f'{today_date}_urls.csv'

df = pd.read_csv(load_name)
urls = []

for x in df['url_id']:
    urls.append("https://www.sreality.cz/api/cs/v2/estates/" + str(x))


# create an empty list to store the data
data_list = []

# loop through the urls and extract the data
for url in urls:
    # send HTTP request to the specified URL and save the response from server in a response object called r
    r = requests.get(url)

    # extracting data in json format
    data = r.json()

    # append the data to the list
    data_list.append(data)


In [5]:
# Create an empty DataFrame
dict_list = []

# loop through the data_list and extract the required data for each item
for data in data_list:
    # create an empty dictionary to store the data for this item
    dict_data = {}

    # check if '_links' and 'self' keys are in the data dictionary
    if '_links' in data and 'self' in data['_links']:
        # add the href value to the dictionary under the name 'url_id'
        dict_data['url_id'] = data['_links']['self'].get('href', None)

    # check if 'items' key is in the data dictionary
    if 'items' in data:
        # extract the required data for this item
        for item in data['items']:
            # add the value to the corresponding key in the dictionary
            dict_data[item['name']] = item['value']

    # check if 'map' key is in the data dictionary
    if 'map' in data:
        # add the latitude and longitude to the dictionary
        dict_data['Latitude'] = data['map'].get('lat', None)
        dict_data['Longitude'] = data['map'].get('lon', None)

    # check if there are any 'poi' items in the data
    if 'poi' in data:
        # loop through each 'poi' item
        for poi in data['poi']:
            # add the distance to the dictionary under the name of the poi
            # note that this will create a new column for each unique poi name
            dict_data[poi['name']] = poi['distance']

    # append the dictionary to the list
    dict_list.append(dict_data)

# convert the list of dictionaries to a DataFrame
df = pd.DataFrame(dict_list)

df.head()


,url_id,Celková cena,Poznámka k ceně,Aktualizace,ID,Stavba,Stav objektu,Vlastnictví,Podlaží,Užitná plocha,...,Druh dražby,Místo konání dražby,Datum konání dražby,Termín 1. prohlídky,Termín 2. prohlídky,Dražební vyhláška,Posudek znalce,Plocha bazénu,Minimální kupní cena,Velikost podílu
0,/cs/v2/estates/722035788,6 832 000,"cena včetně sklepa / parkovací stání 350.000 ,...",Dnes,722035788,Cihlová,Novostavba,Osobní,2. podlaží,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/cs/v2/estates/1669567564,13 254 000,V ceně dvě garážové stání a sklep,Dnes,NaN,Skeletová,Novostavba,Osobní,1. podlaží z celkem 6 včetně 2 podzemních,188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/cs/v2/estates/2418832972,6 361 000,"včetně DPH, včetně poplatků, včetně provize, v...",Dnes,NaN,Cihlová,Novostavba,Osobní,1. podlaží,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/cs/v2/estates/3930260556,2 214 000,"včetně DPH, včetně poplatků, včetně provize, v...",Dnes,NaN,Panelová,Dobrý,Družstevní,5. podlaží včetně 1 podzemního,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/cs/v2/estates/4078400588,17 102 000,"včetně DPH, včetně poplatků, včetně provize, v...",Dnes,NaN,Cihlová,Dobrý,Osobní,3. podlaží,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
filename = f'{today_date}_data.csv'
df.to_csv(filename, index=False)